In [16]:
import pandas as pd

all_features = pd.read_csv('C:/Users/SAMSUNG/Documents/vscode/2024/ML/프로젝트2/data/윈저라이징_inf제거.csv',index_col=0)

# Display the first few rows of the dataset to understand its structure
all_features


,회사명,거래소코드,회계년도,총자본증가율,정상영업이익증가율,순이익증가율,자기자본증가율,매출액증가율,종업원수증가율,매출액총이익률,...,실업률,외환보유액(천달러),CD이자율,환율,수출물가지수증감률,수입물가지수증감률,3년국고채수익률,생산자물가지수,생산자물가지수증감률,부실
0,(주)ES큐브,50120,2012,-16.575097,0.0,92.510776,1.564118,4.854248,-0.4200,28.015517,...,3.1,316897731,2.89,1070.6,-2.367798,-0.724218,2.820,103.31,0.692008,0.0
1,(주)ES큐브,50120,2013,1.849357,0.0,-1427.915750,6.304012,4.692085,-53.5257,24.759975,...,3.2,335647450,2.66,1055.4,-4.268391,-7.917155,2.858,101.66,-1.597135,0.0
2,(주)ES큐브,50120,2014,20.099174,0.0,96.260255,17.123515,-24.126991,-53.5257,21.419545,...,3.6,353600472,2.13,1099.3,-5.961831,-8.166371,2.098,101.12,-0.531182,0.0
3,(주)ES큐브,50120,2015,-1.751499,0.0,-274.898482,0.053133,-5.188115,44.4400,27.803407,...,3.5,358513845,1.67,1172.5,-5.217313,-18.112831,1.662,97.06,-4.015032,0.0
4,(주)ES큐브,50120,2016,89.076046,0.0,1033.661139,44.390095,-18.654872,-23.0800,28.682162,...,3.4,361701404,1.52,1207.7,-3.297016,-4.409925,1.638,95.30,-1.813311,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12317,한화화인케미칼(주),25850,2013,0.000000,0.0,0.000000,0.000000,0.000000,-6.8600,-18.891623,...,3.2,335647450,2.66,1055.4,-4.268391,-7.917155,2.858,101.66,-1.597135,1.0
12318,핸즈코퍼레이션(주),143210,2022,-3.617503,0.0,-21.656986,-25.683177,38.426889,-24.2500,4.960248,...,3.0,399043064,3.98,1264.5,16.641876,20.573937,3.722,115.29,8.375635,1.0
12319,현대로템(주),64350,2015,14.613589,0.0,-1427.915750,-17.312414,3.698453,-1.4100,-0.362243,...,3.5,358513845,1.67,1172.5,-5.217313,-18.112831,1.662,97.06,-4.015032,1.0
12320,현대페인트(주),11720,2012,0.000000,0.0,0.000000,0.000000,0.000000,-1.4500,22.162032,...,3.1,316897731,2.89,1070.6,-2.367798,-0.724218,2.820,103.31,0.692008,1.0


In [3]:
all_features.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12322 entries, 0 to 12321
Data columns (total 75 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   회사명                12322 non-null  object 
 1   거래소코드              12322 non-null  int64  
 2   회계년도               12322 non-null  int64  
 3   총자본증가율             12322 non-null  float64
 4   정상영업이익증가율          12322 non-null  float64
 5   순이익증가율             12322 non-null  float64
 6   자기자본증가율            12322 non-null  float64
 7   매출액증가율             12322 non-null  float64
 8   종업원수증가율            12322 non-null  float64
 9   매출액총이익률            12322 non-null  float64
 10  매출액정상영업이익률         12322 non-null  float64
 11  매출액순이익률            12322 non-null  float64
 12  자기자본정상영업이익률        12322 non-null  float64
 13  자기자본순이익률           12322 non-null  float64
 14  금융비용부담률            12322 non-null  float64
 15  수지비율(관계기업투자손익 제외)  12322 non-null  float64
 16  사내유보 대 자기자본비율      12322 no

In [20]:
all_features = all_features.drop(columns = ['코스닥연수익률 (%)', '코스피연수익률(%)','코스닥변동성', '코스피변동성'])

In [22]:
from sklearn.feature_selection import f_classif
from scipy.stats import ttest_ind
from sklearn.model_selection import train_test_split

# Drop non-feature columns and separate features and target
X = all_features.drop(columns=['회사명', '거래소코드', '회계년도', '부실'])
y = all_features['부실']

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,stratify=y, random_state = 777)

# Perform F-test for feature selection
f_test, _ = f_classif(X_train, y_train)
f_test_scores = pd.Series(f_test, index=X_train.columns)

# Separate the train data into two groups based on the target variable for T-test
group0_train = X_train[y_train == 0]
group1_train = X_train[y_train == 1]

# Perform T-test for feature selection
t_test_scores = {}
for column in X_train.columns:
    t_stat, _ = ttest_ind(group0_train[column], group1_train[column], nan_policy='omit')
    t_test_scores[column] = t_stat

t_test_scores = pd.Series(t_test_scores)
t_test_scores

총자본증가율         3.913010
정상영업이익증가율      4.449521
순이익증가율        11.345770
자기자본증가율        4.403477
매출액증가율         4.228016
                ...    
수출물가지수증감률      4.157332
수입물가지수증감률      2.166732
3년국고채수익률      -4.933564
생산자물가지수       -0.231862
생산자물가지수증감률     2.581262
Length: 67, dtype: float64

In [27]:
# Calculate p-values for t-test
p_values = {}
for column in X_train.columns:
    _, p_val = ttest_ind(group0_train[column], group1_train[column], nan_policy='omit')
    p_values[column] = p_val

p_values_series = pd.Series(p_values)
p_values_series

총자본증가율        9.185341e-05
정상영업이익증가율     8.714199e-06
순이익증가율        1.261792e-29
자기자본증가율       1.078145e-05
매출액증가율        2.381947e-05
                  ...     
수출물가지수증감률     3.251020e-05
수입물가지수증감률     3.028262e-02
3년국고채수익률      8.225260e-07
생산자물가지수       8.166505e-01
생산자물가지수증감률    9.860356e-03
Length: 67, dtype: float64

In [24]:
f_test_scores

총자본증가율         15.311648
정상영업이익증가율      19.798241
순이익증가율        128.726490
자기자본증가율        19.390606
매출액증가율         17.876115
                 ...    
수출물가지수증감률      17.283408
수입물가지수증감률       4.694726
3년국고채수익률       24.340058
생산자물가지수         0.053760
생산자물가지수증감률      6.662911
Length: 67, dtype: float64

In [4]:
# from sklearn.feature_selection import f_classif
# from scipy.stats import ttest_ind

# # Drop non-feature columns and separate features and target
# X = all_features.drop(columns=['회사명', '거래소코드', '회계년도', '부실'])
# y = all_features['부실']

# # Perform F-test for feature selection
# f_test, _ = f_classif(X, y)
# f_test_scores = pd.Series(f_test, index=X.columns)

# # Separate the data into two groups based on the target variable for T-test
# group0 = all_features[all_features['부실'] == 0]
# group1 = all_features[all_features['부실'] == 1]

# # Perform T-test for feature selection
# t_test_scores = {}
# for column in X.columns:
#     t_stat, _ = ttest_ind(group0[column], group1[column], nan_policy='omit')
#     t_test_scores[column] = t_stat

# t_test_scores = pd.Series(t_test_scores)

# # Display the top features based on F-test and T-test scores
# top_features_f_test = f_test_scores.nlargest(10)
# top_features_t_test = t_test_scores.nlargest(10)

# top_features_f_test, top_features_t_test

(매출액순이익률              4726.729780
 총자산순이익률              4581.862684
 수지비율(관계기업투자손익 제외)    4564.186003
 매출액정상영업이익률           4563.319319
 자기자본순이익률             4364.218829
 총자산영업이익률             4117.113899
 자기자본정상영업이익률          3677.456627
 CASH FLOW 대 매출액비율    3314.007916
 CASH FLOW 대 총자본비율    3023.044824
 사내유보 대 자기자본비율        2968.404152
 dtype: float64,
 매출액순이익률              68.751217
 총자산순이익률              67.689458
 매출액정상영업이익률           67.552345
 자기자본순이익률             66.062235
 총자산영업이익률             64.164740
 자기자본정상영업이익률          60.642037
 CASH FLOW 대 매출액비율    57.567421
 CASH FLOW 대 총자본비율    54.982223
 사내유보 대 자기자본비율        54.483063
 총자본투자효율              39.725760
 dtype: float64)

In [8]:
# # Display F-test and T-test scores for all features
# f_test_scores = f_test_scores.sort_values(ascending=False)
# t_test_scores = t_test_scores.sort_values(ascending=False)

# # Create a DataFrame to hold both scores for easy comparison
# feature_scores = pd.DataFrame({
#     'F-test Score': f_test_scores,
#     'T-test Score': t_test_scores
# })

# # Display the scores for all features
# feature_scores

# # feature_scores.to_excel('2024/ML/프로젝트2/피쳐_T-test&F-test.xlsx', index=False)

,F-test Score,T-test Score
1주당 CASH FLOW,168.395577,12.976732
1주당매출액,102.440789,10.121304
1주당순이익,106.939462,10.341154
1주당순자산,113.976262,10.675967
3년국고채수익률,38.350384,-6.192769
...,...,...
코스피변동성,3.607924,-1.899454
코스피연수익률(%),3.669884,-1.915694
타인자본회전률,699.725254,26.452320
현금비율,3.148025,1.774268


In [11]:
# from sklearn.feature_selection import f_classif
# from scipy.stats import ttest_ind

# # Drop non-feature columns and separate features and target
# X = all_features.drop(columns=['회사명', '거래소코드', '회계년도', '부실'])
# y = all_features['부실']

# # Perform F-test for feature selection
# f_test, f_p_values = f_classif(X, y)
# f_test_scores = pd.Series(f_test, index=X.columns)
# f_p_values_series = pd.Series(f_p_values, index=X.columns)

# # Separate the data into two groups based on the target variable for T-test
# group0 = all_features[all_features['부실'] == 0]
# group1 = all_features[all_features['부실'] == 1]

# # Perform T-test for feature selection
# t_test_scores = {}
# t_p_values = {}
# for column in X.columns:
#     t_stat, p_value = ttest_ind(group0[column], group1[column], nan_policy='omit')
#     t_test_scores[column] = t_stat
#     t_p_values[column] = p_value

# t_test_scores = pd.Series(t_test_scores)
# t_p_values_series = pd.Series(t_p_values)

# # Create a DataFrame to hold both scores and p-values for comparison
# feature_scores = pd.DataFrame({
#     'F-test Score': f_test_scores,
#     'F-test p-value': f_p_values_series,
#     'T-test Score': t_test_scores,
#     'T-test p-value': t_p_values_series
# })

# # Display the scores and p-values for all features
# # print(feature_scores)  # Display the first few rows of the results for verification
# feature_scores


,F-test Score,F-test p-value,T-test Score,T-test p-value
총자본증가율,14.456432,1.441309e-04,3.802161,1.441309e-04
정상영업이익증가율,25.786480,3.869221e-07,5.078039,3.869221e-07
순이익증가율,199.954767,4.805913e-45,14.140536,4.805913e-45
자기자본증가율,23.959050,9.965337e-07,4.894798,9.965337e-07
매출액증가율,20.926775,4.818342e-06,4.574579,4.818342e-06
...,...,...,...,...
수출물가지수증감률,28.380848,1.014008e-07,5.327368,1.014008e-07
수입물가지수증감률,10.020320,1.551996e-03,3.165489,1.551996e-03
3년국고채수익률,38.350384,6.099729e-10,-6.192769,6.099729e-10
생산자물가지수,0.014008,9.057875e-01,-0.118356,9.057875e-01


In [28]:
num_significant_features = (p_values_series < 0.05).sum()
num_significant_features

54

In [29]:
significant_features = p_values_series[p_values_series < 0.05].index.tolist()
significant_features

['총자본증가율',
 '정상영업이익증가율',
 '순이익증가율',
 '자기자본증가율',
 '매출액증가율',
 '종업원수증가율',
 '매출액총이익률',
 '매출액정상영업이익률',
 '매출액순이익률',
 '자기자본정상영업이익률',
 '자기자본순이익률',
 '금융비용부담률',
 '수지비율(관계기업투자손익 제외)',
 '사내유보 대 자기자본비율',
 '총자본회전률',
 '타인자본회전률',
 '유동자산회전률',
 '당좌자산회전률',
 '매출채권회전률',
 '순운전자본회전률',
 '종업원1인당 부가가치',
 '노동장비율',
 '총자본투자효율',
 '설비투자효율',
 '1주당순이익',
 '1주당 CASH FLOW',
 '유보율',
 '부가가치율',
 '노동소득분배율',
 '자기자본구성비율',
 '유동비율',
 '재고자산 대 순운전자본비율',
 '부채비율',
 'CASH FLOW 대 부채비율',
 'CASH FLOW 대 차입금비율',
 'CASH FLOW 대 총자본비율',
 'CASH FLOW 대 매출액비율',
 '1주당매출액',
 '1주당순자산',
 '총자산영업이익률',
 '총자산순이익률',
 'CP전년대비증가율',
 '경기선행지수',
 '경기동행지수',
 'GDP대비국가부채비율',
 '장단기금리차(전월차)(%p)',
 '실업률',
 '외환보유액(천달러)',
 'CD이자율',
 '환율',
 '수출물가지수증감률',
 '수입물가지수증감률',
 '3년국고채수익률',
 '생산자물가지수증감률']

In [30]:
significant_features_sorted = p_values_series[p_values_series < 0.05].sort_values()
significant_features_sorted

자기자본정상영업이익률           0.000000e+00
CASH FLOW 대 총자본비율     0.000000e+00
사내유보 대 자기자본비율         0.000000e+00
CASH FLOW 대 매출액비율     0.000000e+00
총자산영업이익률              0.000000e+00
매출액정상영업이익률            0.000000e+00
매출액순이익률               0.000000e+00
총자산순이익률               0.000000e+00
자기자본순이익률              0.000000e+00
수지비율(관계기업투자손익 제외)     0.000000e+00
금융비용부담률              6.740371e-252
총자본투자효율              6.965801e-230
부가가치율                5.811464e-215
CASH FLOW 대 부채비율     2.606540e-181
순운전자본회전률             7.018662e-108
타인자본회전률              3.030616e-104
부채비율                  1.869580e-76
자기자본구성비율              6.499308e-74
종업원1인당 부가가치           1.718541e-58
총자본회전률                1.669651e-48
유보율                   8.077402e-46
유동자산회전률               4.316555e-32
순이익증가율                1.261792e-29
경기동행지수                1.083664e-27
1주당 CASH FLOW         1.317680e-27
외환보유액(천달러)            1.653413e-26
경기선행지수                1.077757e-25
당좌자산회전률               1.654648e-22
1주당순자산              

In [31]:
significant_features_sorted_list = list(significant_features_sorted.index)
significant_features_sorted_list

['자기자본정상영업이익률',
 'CASH FLOW 대 총자본비율',
 '사내유보 대 자기자본비율',
 'CASH FLOW 대 매출액비율',
 '총자산영업이익률',
 '매출액정상영업이익률',
 '매출액순이익률',
 '총자산순이익률',
 '자기자본순이익률',
 '수지비율(관계기업투자손익 제외)',
 '금융비용부담률',
 '총자본투자효율',
 '부가가치율',
 'CASH FLOW 대 부채비율',
 '순운전자본회전률',
 '타인자본회전률',
 '부채비율',
 '자기자본구성비율',
 '종업원1인당 부가가치',
 '총자본회전률',
 '유보율',
 '유동자산회전률',
 '순이익증가율',
 '경기동행지수',
 '1주당 CASH FLOW',
 '외환보유액(천달러)',
 '경기선행지수',
 '당좌자산회전률',
 '1주당순자산',
 '종업원수증가율',
 '1주당매출액',
 '1주당순이익',
 'GDP대비국가부채비율',
 '환율',
 '설비투자효율',
 'CASH FLOW 대 차입금비율',
 '노동소득분배율',
 '매출액총이익률',
 '실업률',
 'CD이자율',
 '3년국고채수익률',
 '장단기금리차(전월차)(%p)',
 '정상영업이익증가율',
 '자기자본증가율',
 '매출액증가율',
 '수출물가지수증감률',
 '노동장비율',
 '총자본증가율',
 '유동비율',
 '생산자물가지수증감률',
 'CP전년대비증가율',
 '매출채권회전률',
 '재고자산 대 순운전자본비율',
 '수입물가지수증감률']